In [3]:
# Imports
import pandas as pd
import numpy as np
import sys

from datetime import datetime as dt

# Scikit-learn
from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.kernel_ridge import *
from sklearn.svm import *
from sklearn.neighbors import *
from sklearn.linear_model import *
from sklearn.naive_bayes import *
from sklearn.metrics import *

import missingno as msno
import prince

import seaborn as sns
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier

In [4]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

# Load

In [5]:
df = pd.read_csv('train.csv')
df.head()

,id,DC024,DC025,DC205,DC206,DC207,DC208,DC209,DC210,DC211,...,DC237f,DC241,DC242,DC244,DC246,DC252,DC270a,DC109,DC142a,DC201
0,1,11,2,16.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,2.0,1,12.0,60.0,Layak Minum
1,2,11,2,31.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,2,31.0,70.0,Layak Minum
2,3,11,2,12.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,4,11.0,75.0,Layak Minum
3,6,11,2,12.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,2.0,NaN,1.0,0.0,1.0,1,11.0,85.0,Layak Minum
4,7,11,2,12.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,2,11.0,65.0,Layak Minum


In [7]:
df['DC201'] = [1 if i == 'Layak Minum' else 0 for i in df['DC201']]

In [8]:
X = df.drop(['id','DC109', 'DC201'], axis = 1)
y = df['DC201']

In [12]:
X.fillna(0, inplace = True)
missing_values_table(X)

Your selected dataframe has 37 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


# Baseline Model

In [9]:
def fit_model(x_train, y_train, estimator, cv):
  avg_fit_time = []
  sd_fit_time = []
  max_fit_time = []

  avg_train_score = []
  sd_train_score = []
  max_train_score = []
  min_train_score = []

  avg_test_score = []
  sd_test_score = []
  max_test_score = []
  min_test_score = []
  
  estimator_cv = []

  for est in estimator:
    print('\nCurrent:', est)
    score = cross_validate(est, x_train, y_train, scoring = 'f1', cv = cv, return_train_score=True, return_estimator = True,
                           n_jobs = -1, verbose = 1)
    
    avg_fit_time.append(np.mean(score['fit_time']))
    sd_fit_time.append(np.std(score['fit_time']))
    max_fit_time.append(np.max(score['fit_time']))

    avg_train_score.append(np.mean(score['train_score']))
    sd_train_score.append(np.std(score['train_score']))
    max_train_score.append(np.max(score['train_score']))
    min_train_score.append(np.min(score['train_score']))

    avg_test_score.append(np.mean(score['test_score']))
    sd_test_score.append(np.std(score['test_score']))
    max_test_score.append(np.max(score['test_score']))
    min_test_score.append(np.min(score['test_score']))

    param = []
    for i in score['estimator']:
      param.append(i.get_params())
    
    estimator_cv.append(param)

  result = pd.DataFrame({
    'model' : estimator,
    'avg_fit_time' : avg_fit_time,
    'sd_fit_time' : sd_fit_time,
    'max_fit_time' : max_fit_time,

    'avg_train_score' : avg_train_score,
    'sd_train_score' : sd_train_score,
    'max_train_score' : max_train_score,
    'min_train_score' : min_train_score,

    'avg_test_score' : avg_test_score,
    'sd_test_score' : sd_test_score,
    'max_test_score' : max_test_score,
    'min_test_score' : min_test_score,

    'estimators' : estimator_cv,
  })

  return(result)

In [10]:
X_base = X
y_base = y

In [73]:
result_base = fit_model(X_base, y_base, [DecisionTreeClassifier(), RandomForestClassifier(),
                                         SVC(), LGBMClassifier(), CatBoostClassifier(),
                                         HistGradientBoostingClassifier(),
                                         AdaBoostClassifier(), SGDClassifier(),
                                         GaussianNB(), KNeighborsClassifier()])
result_base


Current: DecisionTreeClassifier()


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
1172.48s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1172.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1172.54s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1172.54s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1172.59s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1172.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1172.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1172.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1172.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
1172.74s - pydevd: Sending message related to process being replaced


Current: RandomForestClassifier()


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   13.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Current: SVC()


...


Current: GaussianNB()


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Current: KNeighborsClassifier()


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   35.2s finished


,model,avg_fit_time,sd_fit_time,max_fit_time,avg_train_score,sd_train_score,max_train_score,min_train_score,avg_test_score,sd_test_score,max_test_score,min_test_score,estimators
0,DecisionTreeClassifier(),0.359002,0.038508,0.412877,1.000000,0.000000,1.000000,1.000000,0.787778,0.155947,0.942271,0.461074,"[{'ccp_alpha': 0.0, 'class_weight': None, 'cri..."
1,RandomForestClassifier(),5.934677,0.990107,6.339982,0.999998,0.000005,1.000000,0.999983,0.889894,0.071045,0.948545,0.744453,"[{'bootstrap': True, 'ccp_alpha': 0.0, 'class_..."
2,SVC(),80.785078,3.192877,87.723592,0.947712,0.000008,0.947721,0.947704,0.947712,0.000073,0.947784,0.947630,"[{'C': 1.0, 'break_ties': False, 'cache_size':..."
3,LGBMClassifier(),0.816846,0.032770,0.850614,0.964702,0.001156,0.967204,0.963313,0.821505,0.102331,0.949033,0.696399,"[{'boosting_type': 'gbdt', 'class_weight': Non..."
4,<catboost.core.CatBoostClassifier object at 0x...,33.942944,0.111263,34.116718,0.972076,0.001244,0.974712,0.970692,0.794760,0.122567,0.949172,0.637976,"[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]"
5,<catboost.core.CatBoostClassifier object at 0x...,41.496766,1.158037,42.279285,0.972076,0.001244,0.974712,0.970692,0.794760,0.122567,0.949172,0.637976,"[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]"
6,HistGradientBoostingClassifier(),4.702416,0.513557,5.204806,0.963485,0.001307,0.965729,0.961634,0.825662,0.105770,0.948755,0.678615,"[{'categorical_features': None, 'class_weight'..."
7,AdaBoostClassifier(),3.690923,0.022642,3.727228,0.953129,0.001705,0.956331,0.951469,0.797065,0.233361,0.949436,0.156997,"[{'algorithm': 'SAMME.R', 'base_estimator': 'd..."
8,SGDClassifier(),1.546911,0.288913,1.959784,0.920524,0.041012,0.949014,0.833568,0.917287,0.042354,0.953282,0.814455,"[{'alpha': 0.0001, 'average': False, 'class_we..."
9,GaussianNB(),0.145199,0.008360,0.156576,0.855793,0.006671,0.865931,0.844679,0.847343,0.081869,0.934357,0.688163,"[{'priors': None, 'var_smoothing': 1e-09}, {'p..."


# MCA

In [13]:
mca = prince.MCA(
    n_components=20,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=323
)

mca = mca.fit(X)

In [14]:
mca.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,0.119,35.44%,35.44%
1,0.076,22.53%,57.98%
2,0.038,11.29%,69.27%
3,0.036,10.66%,79.93%
4,0.011,3.29%,83.22%
5,0.010,2.91%,86.13%
6,0.004,1.30%,87.43%
7,0.004,1.15%,88.58%
8,0.003,1.02%,89.60%


In [15]:
mca = prince.MCA(
    n_components=10,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=323
)

mca = mca.fit(X)

In [16]:
mca.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,0.119,35.44%,35.44%
1,0.076,22.53%,57.98%
2,0.038,11.29%,69.27%
3,0.036,10.66%,79.93%
4,0.011,3.29%,83.22%
5,0.010,2.91%,86.13%
6,0.004,1.30%,87.43%
7,0.004,1.15%,88.58%
8,0.003,1.02%,89.60%


# Model Advanced

In [17]:
X_mca = mca.transform(X)
y_mca = y

In [85]:
result_mca  = fit_model(X_mca, y_mca, [DecisionTreeClassifier(), RandomForestClassifier(),
                                         SVC(), LGBMClassifier(), CatBoostClassifier(),
                                          HistGradientBoostingClassifier(),
                                         AdaBoostClassifier(), SGDClassifier(),
                                         GaussianNB(), KNeighborsClassifier()])
result_mca


Current: DecisionTreeClassifier()


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Current: RandomForestClassifier()


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   16.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Current: SVC()


...


Current: GaussianNB()


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Current: KNeighborsClassifier()


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   24.4s finished


,model,avg_fit_time,sd_fit_time,max_fit_time,avg_train_score,sd_train_score,max_train_score,min_train_score,avg_test_score,sd_test_score,max_test_score,min_test_score,estimators
0,DecisionTreeClassifier(),0.724972,0.027031,0.781330,1.000000,0.000000,1.000000,1.000000,0.902802,0.034348,0.942064,0.828136,"[{'ccp_alpha': 0.0, 'class_weight': None, 'cri..."
1,RandomForestClassifier(),14.471224,0.206139,14.783230,0.999995,0.000008,1.000000,0.999983,0.936398,0.024214,0.950685,0.865769,"[{'bootstrap': True, 'ccp_alpha': 0.0, 'class_..."
2,SVC(),37.566664,1.929284,39.708331,0.949988,0.000552,0.951446,0.949337,0.946083,0.007421,0.950921,0.924054,"[{'C': 1.0, 'break_ties': False, 'cache_size':..."
3,LGBMClassifier(),0.960385,0.041702,1.073576,0.960443,0.001331,0.963669,0.959185,0.925097,0.047254,0.950824,0.790576,"[{'boosting_type': 'gbdt', 'class_weight': Non..."
4,<catboost.core.CatBoostClassifier object at 0x...,41.392627,0.140716,41.586088,0.968402,0.001000,0.970451,0.967198,0.922448,0.049203,0.949985,0.784431,"[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]"
5,HistGradientBoostingClassifier(),1.811130,0.163870,2.130207,0.958112,0.001675,0.961675,0.956128,0.926140,0.044345,0.950419,0.800069,"[{'categorical_features': None, 'class_weight'..."
6,AdaBoostClassifier(),5.988692,0.041741,6.027009,0.948595,0.000961,0.951203,0.947635,0.940158,0.018914,0.952226,0.885162,"[{'algorithm': 'SAMME.R', 'base_estimator': 'd..."
7,SGDClassifier(),0.129800,0.004609,0.139898,0.947712,0.000008,0.947721,0.947704,0.947712,0.000073,0.947784,0.947630,"[{'alpha': 0.0001, 'average': False, 'class_we..."
8,GaussianNB(),0.042974,0.003600,0.047679,0.906950,0.004724,0.917811,0.899356,0.900975,0.041466,0.939324,0.796463,"[{'priors': None, 'var_smoothing': 1e-09}, {'p..."
9,KNeighborsClassifier(),0.094214,0.007494,0.103918,0.957998,0.000910,0.960383,0.957228,0.917996,0.051176,0.950574,0.770046,"[{'algorithm': 'auto', 'leaf_size': 30, 'metri..."


In [18]:
X = df.drop(['id','DC109', 'DC201', 'DC235'], axis = 1)
y = df['DC201']

In [19]:
X.fillna(0, inplace = True)
missing_values_table(X)

Your selected dataframe has 36 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [20]:
mca = prince.MCA(
    n_components=13,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=323
)

mca = mca.fit(X)

In [21]:
X_2 = mca.transform(X)

# Simple tuning

In [96]:
from scipy.stats import randint, uniform

# define the parameter distributions to search over
param_distribution = {
    'boosting_type': ['gbdt', 'dart', 'goss'],
    'num_leaves': randint(10, 1000),
    'max_depth': [-1, 5, 10, 15, 20, 25],
    'learning_rate': uniform(0.01, 0.5),
    'n_estimators': randint(50, 1000),
    'min_child_samples': randint(1, 200),
    'reg_alpha': uniform(0, 10),
    'reg_lambda': uniform(0, 10),
    'colsample_bytree': uniform(0.5, 0.5),
    'subsample': uniform(0.5, 0.5),
    'subsample_freq': randint(1, 10),
    'min_split_gain': uniform(0, 1),
    'min_child_weight': uniform(0.01, 10),
    'scale_pos_weight': uniform(0.5, 1.5),
}

# create an instance of the LGBMClassifier estimator
estimator = LGBMClassifier(random_state=42)

# create an instance of RandomizedSearchCV
random_search2 = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=param_distribution,
    n_iter=250,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

res_lgbm2 = random_search2.fit(X_2, y)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


30862.20s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.25s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.32s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.43s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.48s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.53s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.67s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
30862.72s - pydevd: Sending message rela

[CV] END boosting_type=goss, colsample_bytree=0.8982714934301164, learning_rate=0.10171739493308189, max_depth=20, min_child_samples=21, min_child_weight=1.5701864044243652, min_split_gain=0.15599452033620265, n_estimators=380, num_leaves=468, reg_alpha=8.661761457749352, reg_lambda=6.011150117432088, scale_pos_weight=1.5621088666940683, subsample=0.5102922471479012, subsample_freq=2; total time=   0.3s
[CV] END boosting_type=goss, colsample_bytree=0.8982714934301164, learning_rate=0.10171739493308189, max_depth=20, min_child_samples=21, min_child_weight=1.5701864044243652, min_split_gain=0.15599452033620265, n_estimators=380, num_leaves=468, reg_alpha=8.661761457749352, reg_lambda=6.011150117432088, scale_pos_weight=1.5621088666940683, subsample=0.5102922471479012, subsample_freq=2; total time=   0.3s
[CV] END boosting_type=goss, colsample_bytree=0.8982714934301164, learning_rate=0.10171739493308189, max_depth=20, min_child_samples=21, min_child_weight=1.5701864044243652, min_split_ga

[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS


[CV] END boosting_type=goss, colsample_bytree=0.8416317594127292, learning_rate=0.31499832889131046, max_depth=10, min_child_samples=81, min_child_weight=2.597799816000169, min_split_gain=0.662522284353982, n_estimators=51, num_leaves=399, reg_alpha=2.079416628681888, reg_lambda=5.677003278199915, scale_pos_weight=0.5469699386833379, subsample=0.9211423872974993, subsample_freq=2; total time=   0.5s
[CV] END boosting_type=goss, colsample_bytree=0.8416317594127292, learning_rate=0.31499832889131046, max_depth=10, min_child_samples=81, min_child_weight=2.597799816000169, min_split_gain=0.662522284353982, n_estimators=51, num_leaves=399, reg_alpha=2.079416628681888, reg_lambda=5.677003278199915, scale_pos_weight=0.5469699386833379, subsample=0.9211423872974993, subsample_freq=2; total time=   0.5s
[CV] END boosting_type=goss, colsample_bytree=0.9916154429033941, learning_rate=0.24338144662398997, max_depth=20, min_child_samples=51, min_child_weight=6.813075385877797, min_split_gain=0.4504

...

[CV] END boosting_type=goss, colsample_bytree=0.8571633178674988, learning_rate=0.24085807190655956, max_depth=-1, min_child_samples=151, min_child_weight=1.2612525714174694, min_split_gain=0.8504716147608377, n_estimators=362, num_leaves=602, reg_alpha=0.3085790430807245, reg_lambda=0.5189529951841088, scale_pos_weight=0.7132556700558433, subsample=0.6066966739878448, subsample_freq=4; total time=   0.3s
[CV] END boosting_type=gbdt, colsample_bytree=0.6808551274297903, learning_rate=0.4846043134352561, max_depth=-1, min_child_samples=123, min_child_weight=7.60005282521931, min_split_gain=0.10251596406647445, n_estimators=882, num_leaves=200, reg_alpha=5.088905555910559, reg_lambda=3.6928077021554104, scale_pos_weight=1.8993872938545082, subsample=0.9137531599366107, subsample_freq=8; total time=   3.1s
[CV] END boosting_type=goss, colsample_bytree=0.8571633178674988, learning_rate=0.24085807190655956, max_depth=-1, min_child_samples=151, min_child_weight=1.2612525714174694, min_split_

[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS


[CV] END boosting_type=goss, colsample_bytree=0.8571633178674988, learning_rate=0.24085807190655956, max_depth=-1, min_child_samples=151, min_child_weight=1.2612525714174694, min_split_gain=0.8504716147608377, n_estimators=362, num_leaves=602, reg_alpha=0.3085790430807245, reg_lambda=0.5189529951841088, scale_pos_weight=0.7132556700558433, subsample=0.6066966739878448, subsample_freq=4; total time=   0.2s
[CV] END boosting_type=dart, colsample_bytree=0.6002801255364915, learning_rate=0.11178918877318021, max_depth=20, min_child_samples=61, min_child_weight=4.384819266082097, min_split_gain=0.882734380673422, n_estimators=391, num_leaves=824, reg_alpha=1.9111870949600263, reg_lambda=3.1008453834275507, scale_pos_weight=1.5975871048463193, subsample=0.8203177964012933, subsample_freq=9; total time=  46.2s
[CV] END boosting_type=gbdt, colsample_bytree=0.6808551274297903, learning_rate=0.4846043134352561, max_depth=-1, min_child_samples=123, min_child_weight=7.60005282521931, min_split_gai

/Users/t-muhammad.zaki/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
355 fits failed out of a total of 1250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
355 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/t-muhammad.zaki/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/t-muhammad.zaki/Library/Python/3.9/lib/python/site-packages/lightgbm/sklearn.py", line 967, in fit
    super().fit(X, _y, sample_weight=sample_weight, init_score=init_score, eval_set=valid_sets,
  File "/Users/t-muhammad.zaki/Library/Python/3.

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(random_state=42), n_iter=250,
                   n_jobs=-1,
                   param_distributions={'boosting_type': ['gbdt', 'dart',
                                                          'goss'],
                                        'colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x144cfdaf0>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x144cf8640>,
                                        'max_depth': [-1,...
                                        'reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x144cf8d30>,
                                        'scale_pos_weight': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x144ca5760>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x144cf8b20>,
                                        'subsample_freq': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x144cd8730>},
                   random_state=42, scoring='f1', verbose=2)

# 1st submission

In [31]:
clf = LGBMClassifier(boosting_type='goss', learning_rate=0.01, max_depth=2,
               min_child_samples=19, min_split_gain=0.1, n_estimators=179,
               num_leaves=62, reg_alpha=1.0, reg_lambda=0.1)
clf.fit(X_2, y)

df_test = pd.read_csv('test.csv')
x_test = df_test.drop(['id','DC109', 'DC235'], axis = 1)
x_id = df_test['id']

x_test = mca.transform(x_test)

y_test_pred = clf.predict(x_test)
#y_test_pred = ['Layak Minum' if i == 1 else 'Tidak Layak Minum' for i in y_test_pred]

df_submission = pd.DataFrame({
    'id' : x_id,
    'DC201' : y_test_pred
})

df_submission

,id,DC201
0,26718,1
1,26802,1
2,41302,1
3,38698,1
4,44257,1
...,...,...
11985,36943,1
11986,33415,1
11987,41998,1
11988,41567,1


In [32]:
df_submission['DC201'].value_counts()

1    11990
Name: DC201, dtype: int64

In [40]:
pd.DataFrame({'pred':clf.predict(X_2)}).value_counts()

pred
1       35973
dtype: int64